<br>
<u>Notebook Eleven</u> | 
<a href=https://leone.gdn/DLL target=_blank>Report</a> | 
<a href=https://github.com/andrealeone/DLL>Repository</a>
<br><br>
<b>Task 2: Query Preprocessing</b><br><br>
Andrea Leone, Alessandro E. Piotti<br>
Deep Learning Lab — University of Trento<br>
December 2021
<hr><br><br>

In [ ]:
import project 

import os
import numpy           as np
import pickle

import torch
import torch.nn        as nn
import torchvision
import networks

from tqdm.notebook import tqdm

project.notebook()

In [ ]:
_, annotations = project.load_resources() 

<hr><br><br>

## Queries processing

<br><br>Load the test and query images

In [ ]:
sets       = [ 'test', 'queries' ] 
irm        = torchvision.io.ImageReadMode.RGB 
images     = dict()

for key in sets: 
    
    path         = '../dataset/{}/'.format(key)
    paths        = sorted([(path + file) for file in os.listdir(path)])
    images[key]  = dict()
    
    for image_file in tqdm(paths):
        
        image_id = image_file.split('/')[-1].split('.')[0]
        tensor   = torchvision.io.read_image( image_file,irm )
        tensor   = tensor.unsqueeze(0).float()
        
        images[key][image_id] = tensor

<br><br>Define the query-encoding network

In [ ]:
vs =    vector_size = len(annotations.columns) 
es = embedding_size = 300 

In [ ]:
encoder    = torchvision.models.resnet50(pretrained=True) 
encoder.fc = nn.Linear(
    in_features=encoder.fc.in_features,
    out_features=embedding_size, bias=True
)

<br><br>Test the encoder

In [ ]:
q = images['queries']['000000'] 
encoder(q).shape

<br><br>Load the test sample-embedding network

In [ ]:
w = './models/siamese_cnn.v4.x30.net' 

model = networks.SiameseCNN() 
model.load_state_dict( torch.load(w) )
model

<br><br>Load and init the attributes super-model

In [ ]:
extractor_src = pickle.load( open('./data/models.v2.pkl', 'rb') ) 
extractor = lambda _input : project.super_model(_input, src=extractor_src) 

<br><br>Test the models pipeline

In [ ]:
img_id = '000016' 

In [ ]:
m = extractor( images['test'][img_id] ) 
m = m.int().detach().numpy()
print( m )

<br><br>Load the pre-computed attributes

In [ ]:
attributes = pickle.load( open('./data/attributes.v2.pkl', 'rb') ) 

In [ ]:
n = attributes['test'][img_id] - 1 
print( n )

In [ ]:
project.compare_image_attributes( 
    image=images['test'][img_id],
    labels=annotations.columns, r1=m, r2=n
)

<hr>

<br><br>Encode the queries

In [ ]:
queries = dict() 

for key in tqdm(list( images['queries'].keys() )): 
    
    queries[key] = encoder( images['queries'][key] )

<br><br>Export the results

In [ ]:
v = '2' 

with open('./data/queries.v{}.pkl'.format(v), 'wb') as file: 
    pickle.dump(queries, file)

<br><br>